In [ ]:
import pandas as pd
import matplotlib.ticker as mtick
pd.set_option("display.max_columns", 500)
pd.set_option("display.max_rows", 5000)

In [ ]:
years = list(range(1926,2023+1))

In [ ]:
all_data = pd.DataFrame(columns=["date", "trr_1_n", "market_cap_usd"])

In [ ]:
#Cap weight top n
for year in years:
    print(year)
    data = pd.read_parquet(f'../../data/processed/csrp/us_data_{year}_processed.parquet', engine="pyarrow", columns=["date", "trr_1_n", "market_cap_usd"])
    data["date"] = pd.to_datetime(data["date"])
    n_largest = data.groupby("date")[["market_cap_usd", "trr_1_n"]].apply(lambda x: x.nlargest(n, columns="market_cap_usd")).reset_index(level=0)
    
    all_data = pd.concat([all_data, n_largest])
    
total_market_cap = all_data.groupby("date")["market_cap_usd"].sum().reset_index()
    
total_market_cap["trr_1_n"] = total_market_cap["market_cap_usd"].pct_change()

top_n_index = total_market_cap.copy()


In [ ]:
top_n_index.to_parquet("../../data/indicators/US/us_top_500.parquet", index=False)

The following index is not used in the thesis

In [ ]:
all_data = pd.DataFrame(columns=["date", "trr_1_n", "market_cap_usd"])

In [ ]:
#Cap weight top n minus top m stocks
for year in years:
    print(year)
    data = pd.read_parquet(f'../../data/processed/csrp/us_data_{year}_processed.parquet', engine="pyarrow", columns=["date", "trr_1_n", "market_cap_usd"])
    data["date"] = pd.to_datetime(data["date"])
    n_largest = data.groupby("date")[["market_cap_usd", "trr_1_n"]].apply(lambda x: x.nlargest(n, columns="market_cap_usd")).reset_index(level=0)
    n_largest_minus_m = n_largest.groupby("date")[["market_cap_usd", "trr_1_n"]].apply(lambda x: x.nsmallest(n-m, columns="market_cap_usd")).reset_index(level=0)
    
    all_data = pd.concat([all_data, n_largest_minus_m])
    
total_market_cap = all_data.groupby("date")["market_cap_usd"].sum().reset_index()
    
total_market_cap["trr_1_n"] = total_market_cap["market_cap_usd"].pct_change()

top_n_index_minus_m = total_market_cap.copy()

In [ ]:
top_n_index_minus_m.plot(x="date", y="market_cap_usd_adjusted", grid=True)

In [ ]:
top_n_index_minus_m.to_parquet("../../data/indicators/US/us_top_500_minus_top_50.parquet", index=False)

In [ ]:
top_n_index_minus_m.plot(x="date", y="market_cap_usd", grid=True)